# Разработка чат бота

## Импорты

In [1]:
import re
import random
import pickle
import numpy as np
import pandas as pd

# Для QA
import string
from functools import lru_cache
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words

from gensim.models import Word2Vec
import annoy

# Парафразер
import torch
from transformers import AutoModelForSequenceClassification, BertTokenizer

# Для бота
from datetime import datetime
import wikipedia

# telegram
from asyncio import Queue
from telegram import Bot
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters

## Настройки

In [2]:
TELEGRAM_TOKEN = 'TOKEN'

MODEL_PATH = './models/w2v/model.w2v'
SPEAKER_PATH = './models/w2v/speaker.ann'
INDEX_PATH = './models/w2v/index_map.pkl'
PARAPHRASER_PATH = './models/paraphraser/model.pkl'
PARAPHRASER_MODEL = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'

## Модель, которая будет отвечать, если вопроса небыло в сценарии
Подготовка модели в файле Boltalka

In [3]:
class QAModel:
    def __init__(self):
        self.index_map = pickle.load(open(INDEX_PATH, 'rb'))
        self.model = Word2Vec.load(MODEL_PATH)
        self.index = annoy.AnnoyIndex(100 ,'angular')
        self.index.load(SPEAKER_PATH)
        
        self.morpher = MorphAnalyzer()
        self.stop_words = set(get_stop_words("ru"))

    @lru_cache(None)
    def lemmatize_word(self, word):
        return self.morpher.parse(word)[0].normal_form
    
    def answer(self, question):
        question = self.preprocess(question)

        n_w2v = 0
        vector = np.zeros(100)
        for word in question:
            if word in self.model.wv:
                vector += self.model.wv[word]
                n_w2v += 1

        if n_w2v > 0:
            vector = vector / n_w2v

        answer_index = self.index.get_nns_by_vector(vector, 1)
        return self.index_map[answer_index[0]]

    def preprocess(self, text, full_preprocessing=True):    
        text = re.sub(r"<\/?\w+>", " ", text) # HTML теги
        text = re.sub(r"([?.!,])", r" \1 ", text)

        if full_preprocessing:
            text = text.lower() # Ответы не приводим в нижний регистр
            text = re.sub(r"\s+", " ", text) # Двойные пробелы, \n т.д.
            words = [word for word in text.split() if word not in self.stop_words]
            words = [self.lemmatize_word(word) for word in words if word]
            words = [word for word in words if len(word) > 2]
        else:
            [word for word in text.split()]

        if len(" ".join(words)) > 3:
            return words
        else:
            return None

qa_model = QAModel()

## Парафразер

In [4]:
class Paraphraser:
    def __init__(self, device="cpu"):
        self._model = AutoModelForSequenceClassification.from_pretrained(PARAPHRASER_MODEL)
        self._tokenizer = BertTokenizer.from_pretrained(PARAPHRASER_MODEL)

        self.to(device)

    def compare(self, text1, text2, return_proba=False):
        batch = self._tokenizer(text1, text2, return_tensors='pt').to(self._device)
        with torch.inference_mode():
            proba = torch.softmax(self._model(**batch).logits, -1).cpu().numpy()

        if return_proba:
            return proba[0]

        return proba[0][0] < proba[0][1]

    def to(self, device):
        self._device = device
        self._model = self._model.to(device)
        return self

paraphraser_model = Paraphraser()

## Напишем "простенького" чатбота на парафразе

In [5]:
class ChatBot:
    def __init__(self):
        self._commands = {}
        self._qa = qa_model
        self._paraphraser = paraphraser_model
        self.last_cmd = ''

    def add(self, command, question: str="", answer: str="", callback=None, precompare=None):
        command = command.strip()

        if not command:
            return ""
        
        if not command in self._commands:
            self._commands[command] = {}
            self._commands[command]["question"] = []
            self._commands[command]["answer"] = []
        
        if question.strip():
            self._commands[command]["question"].append(question.strip())
        
        if answer.strip():
            self._commands[command]["answer"].append(answer.strip())
        
        if callback:
            self._commands[command]["callback"] = callback
        
        if precompare:
            self._commands[command]["precompare"] = precompare

    def answer(self, question):
        result = None
        cmd_key = None

        for key in self._commands.keys():
            for saved_question in self._commands[key]["question"]:
                pc_question = [question,]
                if self._commands[key].get("precompare", None):
                    pc_question = list(self._commands[key]["precompare"](question))
                    if not pc_question or not pc_question[0]:
                        pc_question = [question,]

                if self._paraphraser.compare(pc_question[0], saved_question):
                    cmd_key = key
                    if len(pc_question) > 1:
                        question = pc_question[1]
                    elif pc_question:
                        question = pc_question[0]

                    break
            
            if cmd_key:
                break

        if cmd_key:
            self.last_cmd = cmd_key
            try:
                cmd = self._commands[cmd_key]
                if cmd["answer"]:
                    result = random.choice(cmd["answer"])

                if "callback" in cmd:
                        if not result:
                            result = ''

                        result += cmd["callback"](question)

            except:
                pass
        
        if not result:
            if question:
                self.last_cmd = 'qa'
                result = self._qa.answer(question).replace('\n', ' ').strip()
            else:
                self.last_cmd = 'unknown'
                result = 'Я тебя не понимаю. Попробуй перефразировать'

        return result

In [6]:
chat_bot = ChatBot()

## Заполним сценарии (в идеале хранить бы всё это в БД, но времени мало)
Чем больше вариантов вопросов для каждого сценария будет, тем лучше будет работать Бот.

In [7]:
chat_bot.add("hi", question="Привет", answer="И тебе привет!")
chat_bot.add("hi", question="Здравствуйте", answer="Приветствую!")

chat_bot.add("who_are_you", question="Как тебя зовут?", answer="Искуственный интелект.")

Узнать время

In [8]:
def get_time(question):
    return ' ' + datetime.now().strftime("%H:%M:%S")

chat_bot.add("time", question="Сколько времени?", answer="Сейчас в москве: ", callback=get_time)
chat_bot.add("time", question="Время не подскажешь?", answer="Сейчас на часах ")

Википедия

In [9]:
wikipedia.set_lang("ru")

def wiki_precomp(question):
    words = [word for word in question.split()]
    
    if len(words) > 2:
        if words[0].lower() == "вики":
            return "Вики найти", ' '.join(words[1:])
        
        return ' '.join(words[:2]), ' '.join(words[2:])
    
    return None, question

def wiki(question):
    question = re.sub(r"[\(\)?!,.]", "", question)
    page = wikipedia.page(question)
    text = page.content[:1000]
    final_text = ''
    for sentence in text.split('.')[:-1]:
        if len(sentence.strip()) > 3:
            final_text += sentence + '. '
    
    final_text = final_text.strip()
    return final_text

chat_bot.add("wiki", precompare=wiki_precomp)
chat_bot.add("wiki", question="Вики найти", callback=wiki)
chat_bot.add("wiki", question="Что такое")
chat_bot.add("wiki", question="Кто такой")

Тестируем

In [10]:
chat_bot.answer("Что такое браузер?")

'Бра́узер, веб-обозреватель или веб-браузер (от англ.  web browser, МФА: [wɛb ˈbraʊ. zə(ɹ), -zɚ]; устар.  бро́узер) — прикладное программное обеспечение для просмотра страниц, содержания веб-документов, компьютерных файлов и их каталогов; управления веб-приложениями; а также для решения других задач.  В глобальной сети браузеры используют для запроса, обработки, манипулирования и отображения содержания веб-сайтов.  Многие современные браузеры также могут использоваться для обмена файлами с серверами FTP, а также для непосредственного просмотра содержания файлов многих графических форматов (gif, jpeg, png, svg), аудио- и видеоформатов (mp3, mpeg), текстовых форматов (pdf, djvu) и других файлов. \nЕсть серверы на которых браузер работает.  Серверы из разных стран, к которым браузер подключается и работает. \nФункциональные возможности браузеров постоянно расширяются и улучшаются благодаря конкуренции между их разработчиками и высоким темпам развития и внедрения информационных технологий.

In [11]:
print("Здравия желаю!", "-", chat_bot.answer("Здравия желаю!"))
print("Привет", "-", chat_bot.answer("Привет"))
print("Здарова!", "-", chat_bot.answer("Здарова!")) # Отловил парафразой
print("Как тебя зовут?", "-", chat_bot.answer("Как тебя зовут?"))
print("Сколько времени?", "-", chat_bot.answer("Сколько времени?"))
print("Как погодка?", "-", chat_bot.answer("Как погодка?"))
print("Какой фильм посмотреть?", "-", chat_bot.answer("Какой фильм посмотреть?"))

Здравия желаю! - И тебе привет!
Привет - Приветствую!
Здарова! - Приветствую!
Как тебя зовут? - Искуственный интелект.
Сколько времени? - Сейчас на часах 12:10:58
Как погодка? - а У НАС ПРЕКРАСНО , СОЛНЦЕ СВЕТИТ ЯСНО НАМ .
Какой фильм посмотреть? - зависит от вкусов конечно) хотите тяжёлый и грустный фильм-посмотрите Бен Х или Куда приводят мечты например . . . в фильмах немного разбираюсь) если что нужно вдруг-пишите .


## Telegram

In [12]:
class TelegramBot:
    def __init__(self, token, max_message_len=1000):
        self.updater = Updater(token)
        self.dispatcher = self.updater.dispatcher
        self.chat_bot = chat_bot
        self.max_message_len = max_message_len

    def start_command(self, update, context):
        context.bot.send_message(chat_id=update.message.chat_id, text="Запуск...")
    
    def answer(self, update, context):
        if update.message:
            question = update.message.text
            answer = self.chat_bot.answer(question)[:self.max_message_len]
            chat_id = update.message.chat_id
            context.bot.send_message(chat_id=chat_id, text=answer)
            print(f"Q: {question};\nA: {answer}\ncmd:{self.chat_bot.last_cmd}\n{'_'*10}\n")

        return

    def start(self):
        start_command_handler = CommandHandler('start', self.start_command)
        text_message_handler = MessageHandler(Filters.text, self.answer)
        self.dispatcher.add_handler(start_command_handler)
        self.dispatcher.add_handler(text_message_handler)
        self.updater.start_polling(clean=False)
        self.updater.idle()

telegram_bot = TelegramBot(TELEGRAM_TOKEN)

In [13]:
telegram_bot.start()

Q: Привет;
A: Приветствую!
cmd:hi
__________

Q: Кто ты?;
A: Искуственный интелект.
cmd:who_are_you
__________

Q: Кто такой Есенин?;
A: Серге́й Алекса́ндрович Есе́нин (21 сентября [3 октября] 1895[…], Константиново, Рязанская губерния — 28 декабря 1925[…], Ленинград, СССР) — русский поэт и писатель.  Одна из крупнейших личностей Серебряного века.  Представитель новокрестьянской поэзии и лирики, а в более позднем периоде творчества — имажинизма. 
В разные периоды творчества в его стихотворениях находили отражение социал-демократические идеи, образы революции и Родины, деревни и природы, любви и поиска счастья. 


== Биография ==

Родился Сергей Есенин 3 октября 1895 года в селе Константиново Кузьминской волости Рязанского уезда Рязанской губернии, в крестьянской семье.  Отец — Александр Никитич Есенин (1873—1931), мать — Татьяна Фёдоровна Титова (1875—1955).  Сёстры — Екатерина (1905—1977), Александра (1911—1981), единоутробный брат — Александр Иванович Разгуляев (1902—1961).
cmd:wiki


![Alt text](<Снимок экрана.png>)